# Feature Engineering

Prepare the data frames for modeling.

In [1]:
import duckdb
import pandas as pd
import plotnine as p9
import numpy as np

# Read CSV files using DuckDB and convert to pandas data frames
con = duckdb.connect()

df_alerts = con.execute("SELECT * FROM 'synthetic_alerts.csv'").df()
df_transactions = con.execute("SELECT * FROM 'synthetic_transactions.csv'").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Save the data frames as parquet files (to improve read speed in the next notebook).

# Aggregate the transactions data

Each alert is associated with multiple transactions. Aggregate the transactions to the alert level.

In [2]:
# Create pivoted aggregations for each metric
agg_functions = {
    'Size': ['min', 'mean', 'median', 'max', 'std', 'count', 'sum']
}

# Group by AlertID, Type, and Entry, then aggregate
df_transactions_agg = (df_transactions
    .copy(deep=True)
    .pivot_table(
        index='AlertID',
        columns=['Type', 'Entry'],
        values='Size',
        aggfunc=agg_functions['Size']
    )
    .fillna(0)
    )

# Flatten column names
df_transactions_agg.columns = [
    f"{type_}_{entry}_{stat}Size" 
    for stat, type_, entry in df_transactions_agg.columns
]

# Reset index to make AlertID a regular column
df_transactions_agg = df_transactions_agg.reset_index()

df_transactions_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 57 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   AlertID                          20000 non-null  int64  
 1   Card_Credit_minSize              20000 non-null  float64
 2   Card_Debit_minSize               20000 non-null  float64
 3   Cash_Credit_minSize              20000 non-null  float64
 4   Cash_Debit_minSize               20000 non-null  float64
 5   International_Credit_minSize     20000 non-null  float64
 6   International_Debit_minSize      20000 non-null  float64
 7   Wire_Credit_minSize              20000 non-null  float64
 8   Wire_Debit_minSize               20000 non-null  float64
 9   Card_Credit_meanSize             20000 non-null  float64
 10  Card_Debit_meanSize              20000 non-null  float64
 11  Cash_Credit_meanSize             20000 non-null  float64
 12  Cash_Debit_meanSiz

# Merge the alerts and transactions data

In [3]:
# Merge dataframes
df_full_merged = pd.merge(df_transactions_agg, df_alerts, on='AlertID', how='left')

# Convert Outcome to categorical with Dismiss as first level
df_full_merged['Outcome'] = pd.Categorical(df_full_merged['Outcome'], 
                                    categories=['Dismiss', 'Report'],
                                    ordered=True)

In [4]:
df_full_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 59 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   AlertID                          20000 non-null  int64         
 1   Card_Credit_minSize              20000 non-null  float64       
 2   Card_Debit_minSize               20000 non-null  float64       
 3   Cash_Credit_minSize              20000 non-null  float64       
 4   Cash_Debit_minSize               20000 non-null  float64       
 5   International_Credit_minSize     20000 non-null  float64       
 6   International_Debit_minSize      20000 non-null  float64       
 7   Wire_Credit_minSize              20000 non-null  float64       
 8   Wire_Debit_minSize               20000 non-null  float64       
 9   Card_Credit_meanSize             20000 non-null  float64       
 10  Card_Debit_meanSize              20000 non-null  float64  

# Save data frame to parquet file

Parquet files are easier to read in the next notebook.

In [5]:
df_full_merged.to_parquet('full_merged.parquet')

The next notebook will fit some machine learning models using AutoGluon (an AutoML library) to predict the outcome of the alerts. We will also evaluate the models and plot the results.